In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
import numpy as np
import pandas as pd
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.compat.v1.set_random_seed(42)

In [3]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

Using TensorFlow backend.


#Binary Class

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = 'UCI_HAR_Dataset/'+subset+'/Inertial Signals/'+signal+'_'+subset+'.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

    
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = 'UCI_HAR_Dataset/'+subset+'/y_'+subset+'.txt'
    y = _read_csv(filename)[0]
    y[y<=3] = 0
    y[y>3] = 1

    return pd.get_dummies(y).as_matrix()

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [5]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])

print(timesteps)
print(input_dim)
print(len(X_train))

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

128
9
7352
(7352, 128, 9)
(7352, 2)
(2947, 128, 9)
(2947, 2)


In [6]:
#Training with the best parameters

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(2, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),
          epochs=15)

# Confusion Matrix
score = model.evaluate(X_test, Y_test)
score





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 5,442
Trainable params: 5,442
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 7352 samples, validate on 2947 samples
Epoch 1/15





7352/7352 [==============================] - 21s 3ms/step - loss: 0.3277 - acc: 0.8731 - val_loss: 0

[0.03120530384494252, 0.9888021717000339]

In [0]:
#save model
model.save('model_binary.h5')

#Dynamic and Static classes

In [8]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = 'UCI_HAR_Dataset/'+subset+'/Inertial Signals/'+signal+'_'+subset+'.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

    
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = 'UCI_HAR_Dataset/'+subset+'/y_'+subset+'.txt'
    y = _read_csv(filename)[0]

    return y #pd.get_dummies(y).as_matrix()

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [9]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352,)
(2947, 128, 9)
(2947,)


In [10]:
a=Y_train.tolist()
b=Y_train.argsort()
b=b.tolist()
print(a.count(1))
print(a.count(2))
print(a.count(3))
print(a.count(4))
print(a.count(5))
print(a.count(6))

print("------------------------")

dyn=a.count(1)+a.count(2)+a.count(3)
print(dyn)
sta=a.count(4)+a.count(5)+a.count(6)
print(sta)

print("------------------------")

b_dyn=b[0:3285]
b_sta=b[3285:]
print(len(b_dyn))
print(len(b_sta))


1226
1073
986
1286
1374
1407
------------------------
3285
4067
------------------------
3285
4067


In [0]:
X_train_dyn=[]
X_train_sta=[]
Y_train_dyn=[]
Y_train_sta=[]

for i in b_dyn :
  m=X_train[i]
  X_train_dyn.append(m)

X_train_dyn=np.asarray(X_train_dyn)



for i in b_sta :
  n=X_train[i]
  X_train_sta.append(n)

X_train_sta=np.asarray(X_train_sta)


for i in b_dyn :
  p=Y_train[i]
  Y_train_dyn.append(p)

Y_train_dyn=np.asarray(Y_train_dyn)


for i in b_sta :
  q=Y_train[i]
  Y_train_sta.append(q)

Y_train_sta=np.asarray(Y_train_sta)


In [12]:
a=Y_test.tolist()
b=Y_test.argsort()
b=b.tolist()
print(a.count(1))
print(a.count(2))
print(a.count(3))
print(a.count(4))
print(a.count(5))
print(a.count(6))

print("------------------------")

dyn=a.count(1)+a.count(2)+a.count(3)
print(dyn)
sta=a.count(4)+a.count(5)+a.count(6)
print(sta)

print("------------------------")

b_dyn=b[0:1387]
b_sta=b[1387:]
print(len(b_dyn))
print(len(b_sta))


496
471
420
491
532
537
------------------------
1387
1560
------------------------
1387
1560


In [0]:
X_test_dyn=[]
X_test_sta=[]
Y_test_dyn=[]
Y_test_sta=[]

for i in b_dyn :
  m=X_test[i]
  X_test_dyn.append(m)

X_test_dyn=np.asarray(X_test_dyn)



for i in b_sta :
  n=X_test[i]
  X_test_sta.append(n)

X_test_sta=np.asarray(X_test_sta)


for i in b_dyn :
  p=Y_test[i]
  Y_test_dyn.append(p)

Y_test_dyn=np.asarray(Y_test_dyn)


for i in b_sta :
  q=Y_test[i]
  Y_test_sta.append(q)

Y_test_sta=np.asarray(Y_test_sta)


In [0]:
for n, i in enumerate(Y_train_dyn):
  if i == 1:
    Y_train_dyn[n]=0
  if i == 2:
    Y_train_dyn[n]=1
  if i == 3 :
    Y_train_dyn[n]=2

for n, i in enumerate(Y_test_dyn):
  if i == 1:
    Y_test_dyn[n]=0
  if i == 2:
    Y_test_dyn[n]=1
  if i == 3 :
    Y_test_dyn[n]=2


for n, i in enumerate(Y_train_sta):
  if i == 4:
    Y_train_sta[n]=0
  if i == 5:
    Y_train_sta[n]=1
  if i == 6 :
    Y_train_sta[n]=2

for n, i in enumerate(Y_test_sta):
  if i == 4:
    Y_test_sta[n]=0
  if i == 5:
    Y_test_sta[n]=1
  if i == 6 :
    Y_test_sta[n]=2

from keras.utils import to_categorical

Y_train_dyn = np.array(Y_train_dyn)
Y_test_dyn = np.array(Y_test_dyn )
Y_train_sta = np.array(Y_train_sta)
Y_test_sta = np.array(Y_test_sta)


# one hot encode
Y_train_dyn = to_categorical(Y_train_dyn)
Y_test_dyn  = to_categorical(Y_test_dyn)
Y_train_sta = to_categorical(Y_train_sta)
Y_test_sta = to_categorical(Y_test_sta )



In [15]:
print(X_train_dyn.shape)
print(Y_train_dyn.shape)

print(X_test_dyn.shape)
print(Y_test_dyn.shape)

print(X_train_sta.shape)
print(Y_train_sta.shape)

print(X_test_sta.shape)
print(Y_test_sta.shape)


(3285, 128, 9)
(3285, 3)
(1387, 128, 9)
(1387, 3)
(4067, 128, 9)
(4067, 3)
(1560, 128, 9)
(1560, 3)


In [16]:
#Dynamic class

#Training with the best parameters

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim),return_sequences=True))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(32))
# Adding a dense output layer with sigmoid activation
model.add(Dense(3, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train_dyn,
          Y_train_dyn,
          batch_size=32,
          validation_data=(X_test_dyn, Y_test_dyn),
          epochs=20)

# Confusion Matrix
score = model.evaluate(X_test_dyn, Y_test_dyn)
score

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 31,459
Trainable params: 31,459
Non-trainable params: 0
_________________________________________________________________
Train on 3285 samples, validate on 1387 samples
Epoch 1/20
3285/3285 [==============================] - 23s 7ms/step - loss: 0.9351 - acc: 0.5269 - val_loss: 0.8600 - val_acc: 0.6518
Epoch 2/20
3285/3285 [==============================] - 23s 7ms/s

[0.20508717492719, 0.9733237202595529]

In [0]:
model.save('model_dynamic.h5')

In [18]:
#Training with the best parameters

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim),return_sequences=True))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(3, activation='softmax'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

# Training the model
model.fit(X_train_sta,
          Y_train_sta,
          batch_size=32,
          validation_data=(X_test_sta, Y_test_sta),
          epochs=15)

# Confusion Matrix
score = model.evaluate(X_test_sta, Y_test_sta)
score

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 31,459
Trainable params: 31,459
Non-trainable params: 0
_________________________________________________________________
Train on 4067 samples, validate on 1560 samples
Epoch 1/15
4067/4067 [==============================] - 29s 7ms/step

[0.35382931890365005, 0.8826923076923077]

In [0]:
model.save('model_static.h5')

#Prediction

In [0]:
from keras.models import load_model
import pickle
model_binary = load_model('model_binary.h5')
model_dynamic = load_model('model_dynamic.h5')
model_static = load_model('model_static.h5')

In [61]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = 'UCI_HAR_Dataset/'+subset+'/Inertial Signals/'+signal+'_'+subset+'.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

    
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = 'UCI_HAR_Dataset/'+subset+'/y_'+subset+'.txt'
    y = _read_csv(filename)[0]
    
    return pd.get_dummies(y).as_matrix()


def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [62]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [63]:
#predict binary
a=model_binary.predict_classes(X_test)
b=np.argsort(a)
a=a.tolist()
print(a.count(0))
print(a.count(1))



1408
1539


In [64]:
#separate dynamic and static classes
dyn=b[0:1408]
sta=b[1408:]

print(dyn.shape)
print(sta.shape)

(1408,)
(1539,)


In [0]:
x_test_dyn=[]

for i in dyn :
    a=X_test[i]
    x_test_dyn.append(a)
x_test_dyn=np.asarray(x_test_dyn)

x_test_sta=[]

for i in sta :
    a=X_test[i]
    x_test_sta.append(a)
x_test_sta=np.asarray(x_test_sta)

In [0]:
dyn_pre=model_dynamic.predict_classes(x_test_dyn)
sta_pre=model_static.predict_classes(x_test_sta)

In [67]:
a=sta_pre
a=a.tolist()
print(a.count(0))
print(a.count(1))
print(a.count(2))

406
613
520


In [0]:
for n, i in enumerate(a):
    if i == 0:
        a[n] = 3
    if i == 1:
        a[n]=4
    if i == 2 :
        a[n]=5

In [69]:
print(a.count(3))
print(a.count(4))
print(a.count(5))

406
613
520


In [0]:
sta_pre=a
sta_pre=np.asarray(sta_pre)

In [71]:
print(type(dyn))
print(type(sta))
print(type(dyn_pre))
print(type(sta_pre))

print(dyn.shape)
print(sta.shape)
print(dyn_pre.shape)
print(sta_pre.shape)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1408,)
(1539,)
(1408,)
(1539,)


In [0]:
dyn_zip=list(zip(dyn, dyn_pre))
sta_zip=list(zip(sta,sta_pre))

d_s=dyn_zip+sta_zip

In [0]:
y_pred=[]

for i in range (0,2947):
    y_pred.append(i)

In [0]:
for i in d_s:
    a=i[0]
    b=i[1]
    y_pred[a]=b

In [75]:
from keras.utils import to_categorical

y_pred = np.array(y_pred)
print(y_pred)
# one hot encode
y_pred = to_categorical(y_pred)
print(y_pred)

[4 4 4 ... 1 1 1]
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.9141499830335935